# Classification with Tabular Data using Scikit-Learn and Datalab


In this 5-minute quickstart tutorial, we use cleanlab with scikit-learn models to find potential label errors in a classification dataset with tabular (numeric/categorical) features. Tabular (or *structured*) data are typically organized in a row/column format and stored in a SQL database or file types like: CSV, Excel, or Parquet. Here we study the [German Credit](https://www.openml.org/d/31) dataset which contains 1,000 individuals described by 20 features, each labeled as either "good" or "bad" credit risk. cleanlab automatically shortlists _hundreds_ of examples from this dataset that confuse our ML model; many of which are potential label errors (due to annotator mistakes), edge cases, and otherwise ambiguous examples.

**Overview of what we'll do in this tutorial:**

- Build a simple credit risk classifier with scikit-learn's HistGradientBoostingClassifier.

- Use this classifier to compute out-of-sample predicted probabilities, `pred_probs`, via cross validation.

- Identify potential label errors in the data with cleanlab's `Datalab` class


<div class="alert alert-info">
Quickstart
<br/>
    
Already have (out-of-sample) `pred_probs` from a model trained on an existing set of labels? Run the code below to find any potential label errors in your dataset.

<div  class=markdown markdown="1" style="background:white;margin:16px">  
    
```ipython3 
from cleanlab import Datalab

lab = Datalab(data=your_dataset, label_name="column_name_of_labels")
lab.find_issues(pred_probs=your_pred_probs, issue_types={"label":{}})

lab.get_issues("label")
```
   
</div>
</div>

## 1. Install required dependencies


You can use `pip` to install all packages required for this tutorial as follows:

```ipython3
!pip install sklearn datasets
!pip install cleanlab[datalab]
# Make sure to install the version corresponding to this tutorial
# E.g. if viewing master branch documentation:
#     !pip install git+https://github.com/cleanlab/cleanlab.git
```

In [1]:
# Package installation (hidden on docs website).
dependencies = ["cleanlab", "sklearn", "datasets"]

if "google.colab" in str(get_ipython()):  # Check if it's running in Google Colab
    %pip install cleanlab  # for colab
    cmd = ' '.join([dep for dep in dependencies if dep != "cleanlab"])
    %pip install $cmd
else:
    missing_dependencies = []
    for dependency in dependencies:
        try:
            __import__(dependency)
        except ImportError:
            missing_dependencies.append(dependency)

    if len(missing_dependencies) > 0:
        print("Missing required dependencies:")
        print(*missing_dependencies, sep=", ")
        print("\nPlease install them before running the rest of this notebook.")

In [2]:
import random
import numpy as np
import pandas as pd

from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import HistGradientBoostingClassifier

from cleanlab import Datalab

SEED = 100

np.random.seed(SEED)
random.seed(SEED)

## 2. Load and process the data


We first load the data features and labels (which are possibly noisy).


In [3]:
grades_data = pd.read_csv("https://s.cleanlab.ai/grades-tabular-demo-v2.csv")

X_raw = grades_data[["exam_1", "exam_2", "exam_3", "notes"]]
labels = grades_data["letter_grade"]

Next we preprocess the data, here we apply one-hot encoding to features with categorical data.

In [4]:
cat_features = ["notes"]
X_encoded = pd.get_dummies(X_raw, columns=cat_features, drop_first=True)

num_features = ["exam_1", "exam_2", "exam_3"]
scaler = StandardScaler()
X_processed = X_encoded.copy()
X_processed[num_features] = scaler.fit_transform(X_encoded[num_features])

<div class="alert alert-info">
Bringing Your Own Data (BYOD)?

Assign your data's features to variable `X` and its labels to variable `labels` instead.

</div>

## 3. Select a classification model and compute out-of-sample predicted probabilities


Here we use a simple histogram-based gradient boosting model (similar to XGBoost), but you can choose any suitable scikit-learn model for this tutorial.


In [5]:
clf = HistGradientBoostingClassifier()

To find potential labeling errors, cleanlab requires a probabilistic prediction from your model for every datapoint. However, these predictions will be _overfitted_ (and thus unreliable) for examples the model was previously trained on. cleanlab is intended to only be used with **out-of-sample** predicted probabilities, i.e., on examples held out from the model during the training.

K-fold cross-validation is a straightforward way to produce out-of-sample predicted probabilities for every datapoint in the dataset by training K copies of our model on different data subsets and using each copy to predict on the subset of data it did not see during training. An additional benefit of cross-validation is that it provides a more reliable evaluation of our model than a single training/validation split. We can obtain cross-validated out-of-sample predicted probabilities from any classifier via a simple scikit-learn wrapper:


In [6]:
num_crossval_folds = 5 
pred_probs = cross_val_predict(
    clf,
    X_processed,
    labels,
    cv=num_crossval_folds,
    method="predict_proba",
)

## 4. Use datalab to find label issues


Based on the given labels and out-of-sample predicted probabilities, cleanlab can quickly help us identify poorly labeled instances in our data table. 

Here, we use `Datalab` to find potential label errors in our data. `Datalab` has several ways of loading the data. In this case, we’ll simply wrap the training features and noisy labels in a dictionary. We can instantiate our `Datalab` object with the dictionary created, and then pass in the model predicted probabilities we obtained above, and specify that we want to look for label errors by specifying that using the `issue_types` argument.

For a dataset with N examples from K classes, the labels should be a 1D array of length N and predicted probabilities should be a 2D (N x K) array.

In [7]:
data = {"X": X_processed.values, "y": labels}

lab = Datalab(data, label_name="y")
lab.find_issues(pred_probs=pred_probs, issue_types={"label":{}})

Finding label issues ...
Audit complete. 294 issues found in the dataset.


In [8]:
lab.report()

Here is a summary of the different kinds of issues found in the data:

issue_type    score  num_issues
     label 0.657811         294

(Note: A lower score indicates a more severe issue across all examples in the dataset.)


----------------------- label issues -----------------------

About this issue:
	Examples whose given label is estimated to be potentially incorrect
    (e.g. due to annotation error) are flagged as having label issues.
    

Number of examples with this issue: 294
Overall dataset quality in terms of this issue: : 0.6578

Examples representing most severe instances of this issue:
     is_label_issue  label_score given_label predicted_label
3              True     0.000005           C               F
886            True     0.000059           D               B
709            True     0.000104           F               C
723            True     0.000169           A               C
689            True     0.000181           B               D


From above, we see that `Datalab` has identified many label issues in the data. We can view the quality of all labels and details regarding if a label has been identified as an error using this `get_issues` method.

In [9]:
issue_results = lab.get_issues("label")
issue_results.head()

,is_label_issue,label_score,given_label,predicted_label
0,True,0.000842,C,F
1,False,0.555944,B,B
2,False,0.555944,B,B
3,True,0.000005,C,F
4,True,0.004374,C,D


To view the most severe label issues, we can sort the dataframe above by the `label_score` column (a lower score represents that the label is more likely to be erroneous). 

Let's review some of the most likely label errors:

In [10]:
sorted_issues = issue_results.sort_values("label_score").index

X_raw.iloc[sorted_issues].assign(
    given_label=labels.iloc[sorted_issues], 
    predicted_label=issue_results["predicted_label"].iloc[sorted_issues]
).head()

,exam_1,exam_2,exam_3,notes,given_label,predicted_label
3,0.61,0.94,0.78,NaN,C,F
886,89.00,95.00,73.00,NaN,D,B
709,64.00,70.00,86.00,NaN,F,C
723,53.00,89.00,78.00,NaN,A,C
689,77.00,51.00,70.00,NaN,B,D


The dataframe above show the `given_label` which is the original label that has been identified as a likely error, and also the `predicted_label` which is what cleanlab predicts should be the right label for each example. These examples appear the most suspicious to our model and should be carefully re-examined. Perhaps the original annotators missed something when deciding on the labels for these individuals. 

This is a straightforward approach to visualize the rows in a data table that might be mislabeled. 

In [11]:
# Note: This cell is only for docs.cleanlab.ai, if running on local Jupyter or Colab, please ignore it.

identified_label_issues = issue_results[issue_results["is_label_issue"] == True]
label_issue_indices = [3, 886, 723]  # check these examples were found in label issues
if not all(x in identified_label_issues.index for x in label_issue_indices):
    raise Exception("Some highlighted examples are missing from identified_label_issues.")